Exploritory Data Analysis Notebook to refresh what this data is and what topics I should focus on for this project.

In [5]:
#import modules, packages, and libraries

import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

#import history.csv as working base df.

df = pd.read_csv('../vehicles_us.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [6]:
df.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
25559,10500,2014.0,ram 1500,excellent,6.0,gas,166195.0,automatic,truck,NaN,NaN,2019-01-20,23
33161,6000,2006.0,nissan murano,excellent,NaN,gas,NaN,automatic,SUV,black,1.0,2019-04-11,23
32765,15300,2017.0,toyota sienna,excellent,6.0,gas,102000.0,automatic,mini-van,silver,NaN,2019-01-17,32
46342,3950,2006.0,dodge grand caravan,good,6.0,gas,168928.0,automatic,van,NaN,NaN,2019-03-20,67
28350,11900,2010.0,ford f-150,excellent,8.0,gas,137000.0,automatic,pickup,NaN,1.0,2018-06-06,99
34567,22900,2015.0,ford f-150,excellent,NaN,gas,100500.0,automatic,pickup,silver,1.0,2019-03-10,15
43040,17495,2014.0,jeep cherokee,excellent,6.0,gas,120091.0,automatic,SUV,grey,1.0,2018-05-11,33
18839,5000,2011.0,ford taurus,excellent,4.0,gas,131340.0,automatic,sedan,silver,NaN,2018-11-10,9
18095,12995,2012.0,ram 1500,excellent,8.0,gas,136300.0,automatic,truck,custom,1.0,2018-07-02,29
12135,3400,2006.0,honda civic,good,4.0,gas,165000.0,automatic,coupe,black,NaN,2019-01-18,7


In [9]:
print(df.duplicated().sum())

0


After reviewing the df again and looking around the columns and what interests me. I think I will develop a web app focused on data about how many car sales occur based on paint color choice! So my goal will be to design attractive visuals based on car paint color and demand. I will calculate demand based on two viausal cues: date posted and date sold, which I should be able to determine through some coding utilizing the days_listed column.

In [12]:
#Build initial df with only entries that include the year and the color as these are required for the data. 
colors_df = df.dropna(subset=['paint_color', 'model_year'])
colors_df['model_year'] = colors_df['model_year'].astype(int)
colors_df.head(8)

/var/folders/rt/3pvkzf314n746fyn9dq8p5dh0000gn/T/ipykernel_32292/3802832479.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colors_df['model_year'] = colors_df['model_year'].astype(int)


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
2,5500,2013,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
4,14900,2017,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
5,14990,2014,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
6,12990,2015,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73
7,15990,2013,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68
9,9200,2008,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17
10,19500,2011,chevrolet silverado 1500,excellent,8.0,gas,128413.0,automatic,pickup,black,1.0,2018-09-17,38
11,8990,2012,honda accord,excellent,4.0,gas,111142.0,automatic,sedan,grey,NaN,2019-03-28,29


In [15]:
#Generate a 'date_sold' column by turing the int 'days_listed' into a day data type and add it to the date_posted to retrieve the date_sold date.

#first, check days_listed max amount...
colors_df['days_listed'].unique()

array([ 79,  28,  15,  73,  68,  17,  38,  29,  24, 111,  13,  25,  27,
        64,  39,  40,  23,  42, 101,  20,  14,   7,  31,   1,  36,  21,
        16,  10,  51,  35,  50, 261,  71,   2,  26,  48,  54,  22,  85,
         9,  53,  33,  19,  30,  47,  32,  18,  52, 152,  11, 100,  94,
        82,  44, 141,  70,  57,  62,  46,  96,  58,  93,  49,  55,  67,
        78,  12,   6,  41, 108,  34,  72, 112,  37,  61,  59, 130,   3,
        43,   8,  60,   4,   5, 164,  56,  75,  74,  80,  66,  63, 109,
        81,  90, 127,  45,  88,  91,  77, 114,  84, 110, 140,  69, 104,
       120,  97,  92,  89, 128, 113, 129,  65,  76,  83, 115, 106, 116,
       154, 151, 147, 124,  99, 102, 143,  95,  87,  86, 105, 107,   0,
       103, 155, 118, 149, 144, 117, 122, 132, 137,  98, 263, 148, 179,
       121, 126, 169, 136, 125, 131, 119, 177, 174, 173, 134, 171, 145,
       166, 244, 142, 135, 162, 156, 138, 123, 201, 161, 158, 150, 181,
       184, 133, 139, 146, 183, 163, 206, 236, 157, 159, 204, 23

In [17]:
#What luck. The last value is also the max amount. No values here exceed what is possible for a car to be listed for so we can treat all this data as accurate. 

#Next lets create the day column and add it to an existing column to create another column.
colors_df['days_listed'] = pd.to_timedelta(colors_df['days_listed'], unit='D')
colors_df['date_posted'] = pd.to_datetime(colors_df['date_posted'], format='%Y-%m-%d')
colors_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 39291 entries, 2 to 51523
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   price         39291 non-null  int64          
 1   model_year    39291 non-null  int64          
 2   model         39291 non-null  object         
 3   condition     39291 non-null  object         
 4   cylinders     35293 non-null  float64        
 5   fuel          39291 non-null  object         
 6   odometer      33306 non-null  float64        
 7   transmission  39291 non-null  object         
 8   type          39291 non-null  object         
 9   paint_color   39291 non-null  object         
 10  is_4wd        19447 non-null  float64        
 11  date_posted   39291 non-null  datetime64[ns] 
 12  days_listed   39291 non-null  timedelta64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2), object(6), timedelta64[ns](1)
memory usage: 4.2+ MB


/var/folders/rt/3pvkzf314n746fyn9dq8p5dh0000gn/T/ipykernel_32292/573124930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colors_df['days_listed'] = pd.to_timedelta(colors_df['days_listed'], unit='D')
/var/folders/rt/3pvkzf314n746fyn9dq8p5dh0000gn/T/ipykernel_32292/573124930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colors_df['date_posted'] = pd.to_datetime(colors_df['date_posted'], format='%Y-%m-%d')


In [18]:
#days_listed is now in time format so it can be added to date posted.
colors_df['date_sold'] = colors_df['date_posted'] + colors_df['days_listed']
colors_df.sample(10)

/var/folders/rt/3pvkzf314n746fyn9dq8p5dh0000gn/T/ipykernel_32292/1261446498.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colors_df['date_sold'] = colors_df['date_posted'] + colors_df['days_listed']


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,date_sold
11841,16980,2010,jeep wrangler unlimited,excellent,6.0,gas,NaN,automatic,SUV,custom,1.0,2019-03-13,6 days,2019-03-19
37304,30500,2014,chevrolet silverado 1500,excellent,8.0,gas,45756.0,automatic,truck,white,1.0,2018-09-19,15 days,2018-10-04
41126,10988,2013,chevrolet malibu,like new,4.0,gas,NaN,automatic,sedan,brown,NaN,2018-08-11,13 days,2018-08-24
18054,31000,2018,jeep grand cherokee,like new,NaN,gas,18330.0,automatic,SUV,brown,1.0,2019-04-06,23 days,2019-04-29
15975,10995,2017,hyundai elantra,excellent,4.0,gas,68774.0,automatic,sedan,silver,NaN,2018-05-22,12 days,2018-06-03
35099,12999,2013,toyota corolla,excellent,4.0,gas,74564.0,automatic,sedan,grey,NaN,2019-04-10,20 days,2019-04-30
548,35000,2013,chevrolet silverado,good,8.0,diesel,85000.0,automatic,truck,white,1.0,2018-09-14,17 days,2018-10-01
1974,7000,2009,chevrolet silverado 1500,new,8.0,gas,194700.0,automatic,truck,white,1.0,2018-11-04,17 days,2018-11-21
36180,6700,2005,jeep grand cherokee laredo,excellent,6.0,gas,85000.0,automatic,SUV,grey,1.0,2019-04-15,41 days,2019-05-26
27871,4750,2008,toyota sienna,good,6.0,gas,NaN,automatic,van,red,NaN,2018-05-05,19 days,2018-05-24


In [20]:
#reduce colors_df to include only the data I am interested in

colors_df = colors_df[['model_year', 'paint_color', 'date_posted', 'date_sold']]
colors_df.sample(8)

,model_year,paint_color,date_posted,date_sold
48151,2000,white,2018-06-04,2018-08-18
29698,2004,silver,2018-05-05,2018-06-29
19654,2004,silver,2018-06-04,2018-06-22
23726,2011,blue,2018-11-23,2018-12-13
38097,2013,brown,2018-08-28,2018-12-05
47833,2012,black,2018-07-20,2018-08-18
40292,1999,white,2019-01-15,2019-02-02
16180,2015,white,2018-09-13,2018-10-16


In [21]:
colors_df['paint_color'].unique()

array(['red', 'black', 'white', 'blue', 'grey', 'silver', 'custom',
       'orange', 'yellow', 'brown', 'green', 'purple'], dtype=object)

In [22]:
#Custom sounds fun. Let's leave it in for now. Otherwise these options seem correct and have no typos.
#Lets make an even shorter abbrev for the df we are most interested in...
cdf = colors_df

In [25]:
#Begin organizing for plotting
posted_counts = cdf.groupby(['date_posted', 'paint_color']).size().reset_index(name='count')

fig_posted = px.histogram(posted_counts, x='date_posted', color='paint_color', y='count', title='Cars Listed for Sale by Date and Color')
fig_posted.update_layout(xaxis_title='Date Posted', yaxis_title='Number of Cars')
fig_posted.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed